# Baseline 3
#### Training landmark and transcriptome models together

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import torch

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold

import itertools
import tqdm
import collections

## Prepare data:
#### Read joined data (pre + post treatment)

In [13]:
lincs_joined_df = pd.read_parquet("data/lincs_pretreatment.parquet")
kaggle_joined_df = pd.read_parquet("data/kaggle_pretreatment.parquet")
test_joined_df = pd.read_parquet("data/test_pretreatment.parquet")
print(f"lincs_joined_df = {lincs_joined_df.shape}\nkaggle_joined_df = {kaggle_joined_df.shape}\ntest_joined_df = {test_joined_df.shape}")

lincs_joined_df = (107404, 1842)
kaggle_joined_df = (602, 1841)
test_joined_df = (255, 921)


In [14]:
all_train_df = pd.concat([kaggle_joined_df,lincs_joined_df])
print(f"all_train_df = {all_train_df.shape}")

all_train_df = (108006, 1843)


#### Kaggle provided data

In [15]:
de_train = pd.read_parquet('data/de_train.parquet')
id_map = pd.read_csv('data/id_map.csv',index_col='id')

#### Define features of interest and sort data accordingly.

In [16]:
features = ['cell_type', 'sm_name']
multiindex_features = [("label",'cell_type'),("label",'sm_name')]

transcriptome_cols = de_train.columns[5:]
landmark_cols = kaggle_joined_df["post_treatment"].columns
print(f"transcriptome_cols = {transcriptome_cols.shape}\nlandmark_cols = {landmark_cols.shape}")

transcriptome_cols = (18211,)
landmark_cols = (918,)


In [17]:
# We only need to sort these two dataframes because they represent the same underlying dataset.
de_train = de_train.query("~control").sort_values(features)
kaggle_joined_df = kaggle_joined_df.sort_values(multiindex_features)
# Sanity check that these dfs align.
genes_align = (kaggle_joined_df["post_treatment"] == de_train[landmark_cols]).all(axis=None)
labels_align = (kaggle_joined_df["label"][features] == de_train[features]).all(axis=None)
genes_align and labels_align

True

## Prepare for training
#### Partition data

In [18]:
eval_cells_only_df = kaggle_joined_df[kaggle_joined_df["label"]["cell_type"].isin(["B cells", "Myeloid cells"])].reset_index(drop=True)
mean_score = pd.concat([eval_cells_only_df["label"]["sm_name"],eval_cells_only_df["post_treatment"]],axis=1).groupby("sm_name").agg("mean").mean(axis=1)
classes = np.digitize(mean_score.values, bins=[0,0.1, 0.5, 1])
cpds = mean_score.index.values

# Don't need this df it just looks cool
mean_score.to_frame(name="mean_score").assign(cv_class=classes)

,mean_score,cv_class
sm_name,,
Alvocidib,1.144652,4
CHIR-99021,0.145969,2
Crizotinib,-0.010615,0
Dactolisib,-0.077123,0
Foretinib,-0.007336,0
Idelalisib,0.013947,1
LDN 193189,0.070423,1
Linagliptin,0.033001,1
MLN 2238,2.045447,4


#### CV splits

In [19]:
fold_arr = np.full(len(cpds), -1)
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
for fold, (_, val_ind) in enumerate(skf.split(classes,classes)):
    fold_arr[val_ind] = fold
fold_map = {c: f for c, f in zip(cpds, fold_arr)}
fold_to_cpds = {fold: cpds[fold_arr==fold] for fold in range(n_splits)}

fold_to_cpds

/Users/laurasisson/miniconda3/envs/rdenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


{0: array(['Alvocidib', 'Crizotinib', 'Foretinib', 'LDN 193189', 'R428'],
       dtype=object),
 1: array(['CHIR-99021', 'MLN 2238', 'Palbociclib', 'Penfluridol',
        'Porcn Inhibitor III'], dtype=object),
 2: array(['Dactolisib', 'Idelalisib', 'Linagliptin',
        'O-Demethylated Adapalene', 'Oprozomib (ONX 0912)'], dtype=object)}

In [20]:
identity_transformer = FunctionTransformer()
X = np.array([[0, 1], [2, 3]])
X
identity_transformer.inverse_transform(X), identity_transformer.transform(X)

(array([[0, 1],
        [2, 3]]),
 array([[0, 1],
        [2, 3]]))

#### Model helper functions

In [21]:
def make_transformer(n_components):
    if n_components == 0:
        # Identity transformer
        return FunctionTransformer()
    else:
        return TruncatedSVD(n_components=n_components, random_state=1)

# Unfortunately, sklearn doesn't allow us to use a Ridge model as an intermediate step
# because intermediate steps are supposed to be preprocessors
def make_landmark_model(params):
    params = params["lmk"]
    svd = make_transformer(params["n_comp"])
    model = make_pipeline(ColumnTransformer([('ohe', OneHotEncoder(), list(params["encoded"]))]),
                          Ridge(alpha=params["alpha"], fit_intercept=False))
    return svd, model

def make_transcriptome_model(params):
    params = params["trm"]
    svd = make_transformer(params["n_comp"])
    model = Ridge(alpha=params["alpha"], fit_intercept=False)
    return svd, model

# This changes the models inplace. Returns as a courtesy.
def model_train(mp,x,y):
    svd, model = mp
    z = svd.fit_transform(y)
    model.fit(x, z)
    return svd, model

def model_predict(mp,x):
    svd, model = mp
    d = model.predict(x)
    return svd.inverse_transform(d)
    

#### Draw some example models

In [22]:
example_params = {"lmk":{"n_comp":150,"alpha":.1,"encoded":("sm_name",)},"trm":{"n_comp":150,"alpha":.1}}
print("Landmark")
display(make_landmark_model(example_params))
print("Transcriptome")
display(make_transcriptome_model(example_params))

Landmark


(TruncatedSVD(n_components=150, random_state=1),
 Pipeline(steps=[('columntransformer',
                  ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                                   ['sm_name'])])),
                 ('ridge', Ridge(alpha=0.1, fit_intercept=False))]))

Transcriptome


(TruncatedSVD(n_components=150, random_state=1),
 Ridge(alpha=0.1, fit_intercept=False))

#### Cross Validation function

In [23]:
def crossvalidate(params, print_each=False, print_result=False):
    def train_landmark(cpds):
        mask_va = all_train_df[("label",'cell_type')].isin(['Myeloid cells', 'B cells']) & all_train_df[("label",'sm_name')].isin(cpds)
        validate = all_train_df[mask_va]
    
        mask_tr = ~mask_va
        train = all_train_df[mask_tr]

        svd, model = model_train(make_landmark_model(params), train["label"][features], train["post_treatment"])
        return svd, model

    def train_transcriptome(landmark_mp,cpds):
        mask_va = kaggle_joined_df[("label",'cell_type')].isin(['Myeloid cells', 'B cells']) & kaggle_joined_df[("label",'sm_name')].isin(cpds)
        mask_tr = ~mask_va # 485 or 487 training rows

        train_x = model_predict(landmark_mp, kaggle_joined_df[mask_tr]["label"][features])
        train_y = de_train[mask_tr][transcriptome_cols]

        val_x = model_predict(landmark_mp, kaggle_joined_df[mask_va]["label"][features])
        val_y = de_train[mask_va][transcriptome_cols]

        transcriptome_mp = model_train(make_transcriptome_model(params), train_x, train_y)
        pred_y = model_predict(transcriptome_mp, val_x)
        mrrmse = np.sqrt(np.square(val_y - pred_y).mean(axis=1)).mean()
        return mrrmse
    
    mrrmse_list = []
    for fold, cpds in fold_to_cpds.items():
        landmark_mp = train_landmark(cpds)
        mrrmse = train_transcriptome(landmark_mp,cpds)
        mrrmse_list.append(mrrmse)
        if print_each:
            print(f"# Fold {fold}: {mrrmse:5.3f}")

    mrrmse = np.array(mrrmse_list).mean()
    if print_result:
        print(f"# Overall {mrrmse:5.3f} {params}")
    return mrrmse
                
crossvalidate(example_params, print_each=True)

# Fold 0: 2.012
# Fold 1: 2.666
# Fold 2: 2.172


2.283452586892314

In [24]:
import itertools

alpha_list = [0.1, 1, 5] #10
encoded = [('sm_name',)] #,('cell_type',),('sm_name','cell_type')
n_comps_list = [0,10,30,100]


all_trials = [{"lmk":{"n_comp":lmk_n_comp,"alpha":lmk_alpha,"encoded":encoded},"trm":{"n_comp":trm_n_comp,"alpha":trm_alpha}} for encoded, lmk_alpha, lmk_n_comp, trm_alpha, trm_n_comp in itertools.product(encoded,alpha_list,n_comps_list,alpha_list,n_comps_list)]
print(f"Generated {len(all_trials)} trials.\nExample = {next(iter(all_trials))}")

Generated 144 trials.
Example = {'lmk': {'n_comp': 0, 'alpha': 0.1, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 0, 'alpha': 0.1}}


In [28]:
best_score = float('inf')
best_params = dict()
scores = collections.Counter()
for i,params in enumerate(tqdm.tqdm(all_trials,smoothing=0)):
    score = crossvalidate(params)
    if score < best_score:
        best_score = score
        best_params = params
    scores[i] = -score
for i, score in scores.most_common(5):
    print(all_trials[i],score)
print(f"Best score = {best_score}.\nBest params = {best_params}")

100%|███████████████████████████████████████| 144/144 [1:16:24<00:00, 31.84s/it]

{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 0.1}} -2.259654876047867
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 1}} -2.25968089986029
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 5}} -2.2598107983252733
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 100, 'alpha': 0.1}} -2.260035200635452
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 100, 'alpha': 1}} -2.260060465738705
Best score = 2.259654876047867.
Best params = {'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 0.1}}


## Submission
We retrain the model on the full training data and create a submission file.

In [26]:
lm_mp = model_train(make_landmark_model(best_params), all_train_df["label"][features], all_train_df["post_treatment"])

train_x = model_predict(lm_mp, kaggle_joined_df["label"][features])
train_y = de_train[transcriptome_cols]

trm_mp = model_train(make_transcriptome_model(best_params), train_x, train_y)

Make submission

In [27]:
# Quick sanity check to make sure our submission is ordered correctly.
assert (test_joined_df["label"][features] == id_map[features]).all(axis=None)

# Full pipeline. Nice
test_lm_pred = model_predict(lm_mp, test_joined_df["label"][features])
y_pred = model_predict(trm_mp, test_lm_pred)

y_pred.shape
submission = pd.DataFrame(y_pred, columns=transcriptome_cols, index=id_map.index)
display(submission)
submission.to_csv('submissions/pipeline.csv')

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.485174,0.308513,0.813522,1.103002,0.997349,0.650686,-0.094000,0.643889,-0.359345,0.552053,...,-0.334413,0.098297,0.232372,0.161479,0.742100,0.553216,0.638774,0.527916,-0.299520,0.157486
1,0.056820,0.080760,0.105478,-0.002536,0.117061,0.345525,0.039768,0.132703,0.030165,-0.157085,...,0.020378,0.052416,0.073451,0.198469,0.084084,0.099946,0.036384,0.001365,-0.021197,-0.056184
2,0.341984,0.219168,0.237454,0.042002,0.408000,1.187007,0.059582,0.301075,-0.015984,-0.127638,...,-0.036171,0.115488,0.177245,0.332249,0.361347,0.331133,0.162598,0.085706,-0.034271,-0.090421
3,0.025892,0.036559,0.142437,0.128061,0.132306,0.018028,-0.041141,0.168780,-0.028979,0.122158,...,-0.005573,0.052274,-0.003198,0.115135,0.190370,0.133849,0.157369,0.118075,0.000826,0.026576
4,0.137574,-0.051638,0.071337,0.100727,0.207850,0.174772,-0.102711,0.127906,-0.133145,0.194867,...,-0.190382,0.006200,0.069019,-0.010384,0.163145,0.113274,0.169124,0.137513,0.063657,-0.051418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.241708,-0.203022,-0.207766,-0.202786,-0.411075,-0.498091,-0.087960,-0.147727,-0.044569,0.188796,...,-0.092436,-0.179057,-0.145359,-0.291855,-0.248681,-0.183806,-0.004017,0.017697,-0.011934,-0.048775
251,0.069540,0.023789,0.063895,0.133162,0.118935,0.312506,-0.052441,0.059732,-0.120766,0.311165,...,-0.140056,-0.028260,0.031891,0.018946,0.118167,0.108095,0.156997,0.150959,0.070858,-0.068847
252,-0.084842,-0.067419,0.011255,0.031732,-0.090455,-0.190045,-0.057125,0.027635,-0.066866,0.206733,...,-0.064379,-0.052963,-0.045137,-0.072901,-0.019678,-0.020893,0.101762,0.097841,0.015607,0.015407
